23 October 2018

# UK Gap Analysis

This notebook contains a gap analysis of GBIF Specimens from the United Kingdom.

## 1. Import the required Python libraries

In [ ]:
import numpy as np
import pandas as pd
import reverse_geocoder as rg
from pandas.api.types import CategoricalDtype
%matplotlib inline

## 2. Download GBIF dataset

From the GBIF Occurrence Search page (https://www.gbif.org/occurrence/search), search for all records where:  
`Basis of record = Preserved specimen, Material sample`  
`Country = United Kingdom`  

Download the dataset. Dataset citation: 
> GBIF.org (22 October 2018) GBIF Occurrence Download https://doi.org/10.15468/dl.pdpekf

File saved as **gbif_uk_specimens_20181022.csv**  
1,365,100 records

In [ ]:
gbif_file = 'gbif_uk_specimens_20181022.csv'

## 3. Extract distinct families (+ higher taxa) with valid lat-long coordinates

Import GBIF CSV file (specified above)

In [ ]:
gbif_all = pd.read_csv(gbif_file, sep="\t", dtype='str')

From data, select distinct `kingdom`, `phylum`, `class`, `order`, `family`, `decimalLatitude`, `decimalLongitude`.

In [ ]:
recs = gbif_all[['kingdom', 'phylum', 'class', 'order', 'family', 'decimalLatitude', 'decimalLongitude']].copy()
recs = recs.fillna('')
recs = recs.drop_duplicates()

Filter the data; select only records with numeric lat/long values, and remove records with no taxonomic information.

In [ ]:
def numericcoords(x):
    for coord in [x['decimalLatitude'], x['decimalLongitude']]:
        if coord == '':
            return False
        try:
            pd.to_numeric(coord)
        except ValueError:
            return False
    return True

recs['valid'] = recs.apply(lambda x: numericcoords(x), axis=1)
recs = recs[recs.valid == True]
recs = recs.loc[:, 'kingdom':'decimalLongitude']

blanktaxa = (recs['kingdom'] == '') & \
    (recs['phylum'] == '') & \
    (recs['class'] == '') & \
    (recs['order'] == '') & \
    (recs['family'] == '')
recs = recs[~blanktaxa]
print('{} records with valid coordinates found'.format(recs.shape[0]))

In [ ]:
recs.head()

Export the filtered data file so that it can be added to Dropbox. (The original GBIF file is too huge!)

In [ ]:
recs_file = 'gbif_uk_families.csv'

In [ ]:
recs.to_csv(recs_file, index=False, sep='\t')

## 4. Geocode records and filter out non-UK records

(Re-)import the GBIF records from the file exported in the last step.

In [ ]:
recs = pd.read_csv(recs_file, sep='\t', dtype='str')

Convert string columns `decimalLatitude` and `decimalLongitude` to numeric columns `latitude` and `longitude` respectively.

In [ ]:
recs['decimalLatitude'] = pd.to_numeric(recs['decimalLatitude'])
recs['decimalLongitude'] = pd.to_numeric(recs['decimalLongitude'])
recs = recs.rename(columns={'decimalLatitude': 'latitude', 'decimalLongitude': 'longitude'})

Rough plot of `latitude` and `longitude`

In [ ]:
plotdata = {'latitude': recs.latitude, 'longitude': recs.longitude}
coordmap = pd.DataFrame(data=plotdata)
coordmap.plot.scatter(x='longitude', y='latitude');

Although these samples are supposed to be from the United Kingdom only, coordinates are present all around the globe. Highest concentrations seem to be in Western Europe, Australia, and the Southern Atlantic/Antarctica.

### 4a. reverse_geocoder

GeoNames.org is a website that provides reverse-geocoding of lat/long coordinates to country names, etc. There is a Python library, `reverse_geocoder` (imported as `rg`), that allows users to query the GeoNames database without making API calls.

*(Note, the latitude and longitude returned from rg are not the same as the ones supplied. reverse_geocoder finds the nearest city to the coordinates supplied, then provides the coordinates of that city. Don't use the returned lat/lon as the ones provided.)*

reverse_geocoder requires a (latitude, longitude) tuple to geocode. Added a column, `coordinate`, containing this tuple.

In [ ]:
recs['coordinate'] = recs.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
recs.head()

For efficiency, isolate a list of unique coordinates (`ucoords`) to be reverse geocoded.

In [ ]:
# https://gist.github.com/bsweger/e5817488d161f37dcbd2
ucoords = pd.unique(recs.coordinate.ravel()).tolist()
print(len(ucoords), "unique coordinates")

Reverse geocode the list of ucoords to obtain the country code, administrative district 1, and administrative district 2. Then add the ucoord back to the result. Create a DataFrame, `geolocs`, from the rg results. For ease of joining in the next step, the ucoord is set as the index for this df.

In [ ]:
results = rg.search(ucoords)
gl = []

for r in results:
    x = {}
    x['cc'] = r['cc']
    x['admin1'] = r['admin1']
    gl.append(x)

for x in range(len(gl)):
    gl[x]['lat_lon'] = ucoords[x]
      
geolocs = pd.DataFrame.from_records(gl, columns = ['lat_lon', 'cc', 'admin1'], index='lat_lon')
geolocs.head()

### 4b. Join geolocs to recs to obtain country code and admin1 for each taxon

In [ ]:
georecs = recs.join(
    geolocs, 
    on = 'coordinate', 
    how = 'left'
)
georecs.head()

Filter records to just those from Great Britain (GB; includes England, Northern Ireland, Scotland, and Wales) and Ireland (IE)

In [ ]:
uk = georecs[(georecs['cc'] == 'GB') | (georecs['cc'] == 'IE')].copy()

Determine the `admin1` areas found in Ireland.

In [ ]:
gb_admin1 = sorted(list(georecs[georecs['cc'] == 'GB'].admin1.unique()))
ie_admin1 = sorted(list(georecs[georecs['cc'] == 'IE'].admin1.unique()))
print('GB admin1 areas: {}'.format(gb_admin1))
print('IE admin1 areas: {}'.format(ie_admin1))

### 4c. Assign `country` value to each record

For records from Great Britain, the admin1 area is the actual country name (England, Northern Ireland, Scotland, Wales). For records from Ireland, the admin1 area is the county (Connacht, Leinster, Munster, Ulster). I would like to create a map at the country level, so I am going to add a column `country` that translates records to one of five countries: England, Northern Ireland, Scotland, Wales, and Ireland.

In [ ]:
def assigncountry(admin1):
    if admin1 in gb_admin1:
        return admin1
    elif admin1 in ie_admin1:
        return 'Ireland'
    

uk['country'] = uk['admin1'].apply(assigncountry)
uk.head(10)

Clean up DataFrame - remove redundant `coordinate` column, move `country`.

In [ ]:
uk = uk[['kingdom', 'phylum', 'class', 'order', 'family', 
         'country', 'latitude', 'longitude', 'cc', 'admin1']]
uk.head()

Save geocoded data to a CSV file

In [ ]:
uk_taxa_file = 'uk_taxa_geos.csv'

In [ ]:
uk.to_csv(uk_taxa_file, sep = '\t', index=False)

## 5. Counts of KPCOFG in each country

Re-import country data from CSV file

In [ ]:
uk = pd.read_csv(uk_taxa_file, sep='\t', dtype='str')
uk['latitude'] = pd.to_numeric(uk['latitude'])
uk['longitude'] = pd.to_numeric(uk['longitude'])
uk.head()

Calculate total counts of KPCOF in UK/Ireland

In [ ]:
counts_total = uk[['kingdom', 'phylum', 'class', 'order', 'family']].agg(pd.Series.nunique)
counts_total

Calculate total counts of KPCOF by country

In [ ]:
counts_country = uk[['country', 'kingdom', 'phylum', 'class', 'order', 'family']] \
                   .groupby('country') \
                   .agg(pd.Series.nunique)
counts_country

## 6. GGBN

For each of the countries, we want an idea of how many of their KPCOF are currently in GGBN versus not. 

GGBN data are available through the GGBN Data Portal API. A Gist on querying the API is available here: https://gist.github.com/amdevine/b21ca15fcfaac5c1e75fc33fdcde4056. I use a Python script to retrieve the data and save it as a text file.

Import latest GGBN download file, cleaned and standardized to the CoL backbone taxonomy.

In [ ]:
ggbn_file = 'GGBN Download 2018-10-01.tsv'

In [ ]:
ggbn_all = pd.read_csv(ggbn_file, sep='\t', dtype='str').fillna('xxxxxxxxxxxxxx')
ggbn = ggbn_all.loc[:, 'kingdom':'family'] \
               .drop_duplicates() \
               .sort_values(['kingdom', 'phylum', 'class', 'order', 'family'])
ggbn.head()

Create a second table containing whether the KPCOF are in GGBN (`True`) or not (`False`)

In [ ]:
inggbn = uk.loc[:, 'kingdom':'country']
inggbn['k'] = uk['kingdom'].isin(ggbn.kingdom)
inggbn['p'] = uk['phylum'].isin(ggbn.phylum)
inggbn['c'] = uk['class'].isin(ggbn['class'])
inggbn['o'] = uk['order'].isin(ggbn['order'])
inggbn['f'] = uk['family'].isin(ggbn.family)
inggbn.head()

Calculated the total number of families in the UK/Ireland that are in GGBN

In [ ]:
total_ggbn_families = inggbn[['family', 'f']].drop_duplicates()
total_ggbn_families = sum(total_ggbn_families.f)
print('There are {} families in UK/Ireland that are in GGBN.'.format(total_ggbn_families))

Create summary table with counts of KPCOF in GGBN for each country

In [ ]:
kingdoms = inggbn[['country', 'kingdom', 'k']].drop_duplicates()
kingdoms = kingdoms[['country', 'k']].groupby('country').agg(sum)
phyla = inggbn[['country', 'phylum', 'p']].drop_duplicates()
phyla = phyla[['country', 'p']].groupby('country').agg(sum)
classes = inggbn[['country', 'class', 'c']].drop_duplicates()
classes = classes[['country', 'c']].groupby('country').agg(sum)
orders = inggbn[['country', 'order', 'o']].drop_duplicates()
orders = orders[['country', 'o']].groupby('country').agg(sum)
families = inggbn[['country', 'family', 'f']].drop_duplicates()
families = families[['country', 'f']].groupby('country').agg(sum)

In [ ]:
in_ggbn = pd.merge(kingdoms, phyla, on='country')
in_ggbn = pd.merge(in_ggbn, classes, on='country')
in_ggbn = pd.merge(in_ggbn, orders, on='country')
in_ggbn = pd.merge(in_ggbn, families, on='country')
in_ggbn.columns = ['kingdom', 'phylum', 'class', 'order', 'family']
in_ggbn = pd.DataFrame(in_ggbn.to_records())
in_ggbn

"Un-pivot" the GGBN data into one column, to be added to results later

In [ ]:
in_ggbn = pd.melt(in_ggbn, 
                  id_vars=['country'], 
                  value_vars=['kingdom', 'phylum', 'class', 'order', 'family'],
                  var_name='taxrank',
                  value_name='countggbn',
                 ).drop_duplicates()
cattype = CategoricalDtype(categories=['kingdom', 'phylum', 'class', 'order', 'family'], ordered=True)
in_ggbn['taxrank'] = in_ggbn['taxrank'].astype(cattype)
in_ggbn['countggbn'] = in_ggbn['countggbn'].astype('int32', copy=False)
in_ggbn = in_ggbn.sort_values(['country', 'taxrank'])
in_ggbn

## 7. Catalogue of Life

We also want to look at the percent of life on Earth available in the UK and Ireland. For our backbone taxonomy, we are using the **Catalogue of Life, 24 September 2018** download available from the Darwin Core Archive Export (http://www.catalogueoflife.org/DCA_Export/archive.php) repository.

After light processing of this download, we are left with a file, **CoL Genera 2018-09-24.tsv**

In [ ]:
col_file = 'CoL Genera 2018-09-24.tsv'

In [ ]:
col_all = pd.read_csv(col_file, sep='\t', dtype='str').fillna('')

In [ ]:
col = col_all.loc[:, 'kingdom':'family'] \
             .drop_duplicates() \
             .sort_values(['kingdom', 'phylum', 'class', 'order', 'family'])

Found the number of unique names for each taxonomic rank

In [ ]:
col_counts = col.agg(pd.Series.nunique)
col_counts

## 8. Percentages of CoL taxa in each country

"Unpivot" the `uk` table. Leave country as a grouping variable, but gather all the taxonomic name columns into two columns: one column contains the taxonomic rank (`taxrank`), the other contains the taxonomic name (`taxname`).

In [ ]:
taxcountries = pd.melt(uk, 
                       id_vars=['country'], 
                       value_vars=['kingdom', 'phylum', 'class', 'order', 'family'],
                       var_name='taxrank',
                       value_name='taxname',
                      ).drop_duplicates()
cattype = CategoricalDtype(categories=['kingdom', 'phylum', 'class', 'order', 'family'], ordered=True)
taxcountries['taxrank'] = taxcountries['taxrank'].astype(cattype)
taxcountries.head(10)

Aggregate the unpivoted data by country name and taxonomic rank, counting the unique elements for each country and taxrank.

In [ ]:
tax_counts = taxcountries.groupby(['country', 'taxrank']).agg(pd.Series.nunique)
tax_counts = pd.DataFrame(tax_counts.to_records())
tax_counts = tax_counts.rename(columns={'taxname': 'ukcount'})
tax_counts

Add the total taxrank counts found for the Catalogue of Life (`col_counts`)

In [ ]:
tax_counts['colcount'] = tax_counts['taxrank'].apply(lambda x: col_counts[x])
tax_counts

Create a column containing the percent of all CoL families found in that country and at that taxonomic rank in the UK and Ireland

In [ ]:
tax_counts = tax_counts.assign(ukpercent=tax_counts.ukcount/tax_counts.colcount)
tax_counts

Create columns containing the counts and percents of taxa in GGBN

In [ ]:
tax_counts = pd.merge(tax_counts, in_ggbn, on=['country', 'taxrank'])
tax_counts = tax_counts.assign(percent_not_ggbn=(tax_counts.ukcount-tax_counts.countggbn)/tax_counts.ukcount)
tax_counts

Export table with counts to TSV file

In [ ]:
counts_file = 'uk_taxa_counts.tsv'

In [ ]:
tax_counts.to_csv(counts_file, sep='\t', index=False)

## Results

Produced a map of UK/Ireland in Inkscape.

![](uk_ireland_map_small.png)